In [ ]:

# face detection with mtcnn on a photograph
from matplotlib import pyplot
from mtcnn import MTCNN
# load image from file
filename = 'kusum_dp.jpg'
pixels = pyplot.imread(filename)
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
faces = detector.detect_faces(pixels)
for face in faces:
	print(face)

d:\Anaconda3\envs\gpu_enabled\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
d:\Anaconda3\envs\gpu_enabled\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
d:\Anaconda3\envs\gpu_enabled\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
d:\Anaconda3\envs\gpu_enabled\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 

In [4]:
import tensorflow
import keras

print(tensorflow.__version__, keras.__version__)

2.6.0 2.6.0
